<a href="https://colab.research.google.com/github/erfanera/SlumImprovment/blob/main/demo/pyngrok_Dharavi_leaf_map_URL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyngrok

In [ ]:
!pip install streamlit pyngrok geopandas leafmap

In [30]:
import os
from pyngrok import ngrok
import streamlit as st
import leafmap.foliumap as leafmap
import geopandas as gpd
from shapely.geometry import Polygon
from PIL import Image
import numpy as np

# Set authentication token
ngrok.set_auth_token("2fugZh9AN73a2NEVIPHPnVwXbID_5C42VDP8LV4sQsCsQ6eto")

# Create GeoDataFrame
def create_geodataframe_from_bbox(min_point, max_point):
    rectangle = Polygon([min_point, (max_point[0], min_point[1]), max_point, (min_point[0], max_point[1]), min_point])
    gdf = gpd.GeoDataFrame([1], geometry=[rectangle], columns=['dummy'])
    return gdf

# Split image into grid
def split_image_into_grid(image_path, grid_size):
    image = Image.open(image_path)
    img_array = np.array(image)
    height, width = img_array.shape[:2]
    cell_height = height // grid_size
    cell_width = width // grid_size
    grid_images = []
    for i in range(grid_size):
        for j in range(grid_size):
            top = i * cell_height
            bottom = (i + 1) * cell_height
            left = j * cell_width
            right = (j + 1) * cell_width
            grid_cell = image.crop((left, top, right, bottom))
            grid_images.append(grid_cell)
    return grid_images

def main():
    st.title("Custom Bounding Box and Grid Generator")

    # Map and drawing tool
    m = leafmap.Map(draw_control=True, height=500)
    m.add_basemap("OpenStreetMap")
    m.to_streamlit(height=600)

    # User input for longitude and latitude coordinates
    min_lon = st.number_input("Enter minimum longitude:", value=72.8488)
    min_lat = st.number_input("Enter minimum latitude:", value=19.0384)
    max_lon = st.number_input("Enter maximum longitude:", value=72.8604)
    max_lat = st.number_input("Enter maximum latitude:", value=19.0484)

    # Create bounding box and display map
    gdf = create_geodataframe_from_bbox((min_lon, min_lat), (max_lon, max_lat))
    m.add_gdf(gdf, layer_name="Bounding Box")

    # Extract image
    image_path = st.text_input("Enter path to image file:", "/root/satellite.tif")
    grid_size = st.number_input("Enter grid size (number of grids):", value=10)
    if st.button('Process Image'):
        grid_images = split_image_into_grid(image_path, grid_size)
        st.write("Here are the grid images:")
        for i, grid_image in enumerate(grid_images):
            st.image(grid_image, caption=f"Grid {i}")

    # Establish ngrok connection
    public_url = ngrok.connect(addr='8501')
    st.write('Streamlit is running at URL:', public_url)

if __name__ == "__main__":
    main()


In [27]:
# Start the Streamlit application
os.system("streamlit run app.py &")

# Establish ngrok connection with the correct parameters
public_url = ngrok.connect(addr='8501')
print('Streamlit is running at URL:', public_url)


Streamlit is running at URL: NgrokTunnel: "https://5d90-34-80-116-63.ngrok-free.app" -> "http://localhost:8501"


In [28]:
!ngrok http 8501 &

http - start an HTTP tunnel

USAGE:
  ngrok http [address:port | port] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --domain mydomain.com                           # run ngrok with your own custom domain
  ngrok http 80 --allow-cidr 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your a

In [29]:
!cat /content/ngrok.log

cat: /content/ngrok.log: No such file or directory
